In [1]:
import os, random, sys, time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

In [2]:
file = open('input/login.txt')
lines = file.readlines()
username = lines[0]
password = lines[1]

In [3]:
link = 'https://app.hubspot.com/live-messages/'
file = open('input/messages.txt')
message = file.read()
file1 = open('input/subject.txt')
subject = file1.read()
print('Subject: ' + subject)
print('Messages: ' + message)
data = pd.read_csv("input/data.csv")
data[:]

Subject: Test Subject @customer
Messages: Test Messages @customer


,name,email_adress
0,Test1,test1@gmail.com
1,Test2,test2@gmail.com


In [10]:
def check_exists_by_class(_class):
    try:
        browser.find_element_by_class_name(_class)
    except NoSuchElementException:
        return False
    return True

def check_exists_by_path(_path):
    try:
        browser.find_element_by_xpath(_path)
    except NoSuchElementException:
        return False
    return True

def custom_subject(subject, name):
    return subject.replace('@customer', name)
    
def custom_message(message, name):
    return message.replace('@customer', name)

In [16]:
import os, random, sys, time
from selenium import webdriver
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

log = ''

#login hubspot
file = open('input/login.txt')
lines = file.readlines()
username = lines[0]
password = lines[1]

# read data
file = open('input/messages.txt')
message = file.read()
file1 = open('input/subject.txt')
subject = file1.read()
data = pd.read_csv("input/data.csv")

#Open link on chrome
browser = webdriver.Chrome('driver/chromedriver')
#Access account
hubspot_link = 'https://app.hubspot.com/live-messages/'
browser.get(hubspot_link)
time.sleep(7)
browser.find_element_by_id('username').send_keys(username)
browser.find_element_by_id('password').send_keys(password)
browser.find_element_by_xpath('//button[@type="submit"]').click()
time.sleep(random.uniform(5, 7))
browser.find_element_by_class_name('CollapsableSidebarHeaderDeadZone-p4vqop-0').click()
visitedUsers = pd.read_csv('output/visited_user.csv')
for _,rows in data.iterrows():
    try:
        email_address = rows['email_adress']
        name = rows['name']
        if email_address in visitedUsers.values:
            log += "Already sent to: {}".format(email_address)
        else:
            browser.find_element_by_xpath('//button[@data-test-id="compose-email-button"]').click()
            time.sleep(random.uniform(3, 4))
            subjectCustom = custom_subject(subject,name)
            messageCustom = custom_message(message,name)
            element = browser.find_element_by_xpath('//span[contains(@class, "Select-multi-value-wrapper")]/div[2]/input')
            element.send_keys(email_address)
            time.sleep(2)
            element.send_keys(Keys.ENTER)
            browser.find_element_by_xpath('//input[@data-selenium-test="email-subject-input"]').send_keys(subjectCustom)
            browser.find_element_by_xpath('//div[contains(@class, "rich-text-editor-input")]/div/div[1]/div/div/div/div/div').send_keys(messageCustom)
            time.sleep(1)
            browser.find_element_by_xpath('//button[contains(@data-selenium-test, "rich-text-editor-controls__save-btn")]').click()
            log += "Sent to: {}".format(email_address)
            visitedUsers = visitedUsers.append({'email_address':email_address, 'status':'sent'}, ignore_index = True)
            visitedUsers.to_csv('output/visited_user.csv',index=False) 
            time.sleep(3)
    except:
        log += "Error sent to: {}".format(email_address)
        visitedUsers = visitedUsers.append({'email_address':email_address, 'status':'error'}, ignore_index = True)
        visitedUsers.to_csv('output/visited_user.csv',index=False) 
    print(log)


Sent to: test1@gmail.com
Sent to: test1@gmail.comSent to: test2@gmail.com
